In [1]:
from pyspark.sql import SparkSession

## Constants

In [2]:
data_path = "/app/data/binhluan.csv"

## Init spark session

In [3]:
spark = (
    SparkSession.builder.appName("PySpark Demo")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/17 17:35:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Demo

In [8]:
comments = spark.read.option("header", "true").csv(data_path)

In [9]:
comments.show()

+--------------------+--------------------+------------+--------------+------------+--------------------+---------------+
|                 _c0|                User|comment_date|comment_Rating|Comment_Like|      Comment_Review|Comment_Picture|
+--------------------+--------------------+------------+--------------+------------+--------------------+---------------+
|                   1|Hoa Thanh Trung (...|   7/18/2012|             5|           0|                NULL|             no|
|                   2|    Văn Thắng Nguyễn|   11/7/2012|             5|           0|                NULL|             no|
|                   4|      Trung Trần Mẫn|  11/20/2012|             5|           0|                NULL|             no|
|                  19|        Trần- Nguyễn|   4/20/2014|             5|           0|                NULL|             no|
|                  21|             Minh Le|   5/12/2014|             5|           0|                NULL|             no|
|                  22|  